#Deep Learning Architectures Assignment 2
The goal is to design and train a neural network for a regression task.
Detailed requirements were presented during the last lecture.

#Summary
#Dataset
* The dataset contains 2210 examples.
* Each example consists of 2500 time steps.
* In each time step, 8 values are recorded.
* The result consists of two real numbers representing the coordinates.

##TODO:
* Split the dataset into three sets: test, validation, and training.
* Choose an architecture and design the model.
* Design the training procedure.
* Present the results. \\
// A correctly trained model should make predictions with an average error of < 2.

##What will be evaluated:
* Understanding of the topic, exploration of the dataset, justification of architecture choice [5 points]
* Correctness of the training implementation [5 points]
* Error obtained on test and validation data [5 points]
* Presentation of the achieved results [5 points]


##Extended task for extra points:
* Design a model for noisy data.
* To add noise, use the addNoise function.
* Start tests with low noise: 0.01 or 0.001.

In [97]:
from urllib.request import urlopen
import pickle
import os

def download_part(filename):
  base_url = f"https://github.com/pa-k/AGU/blob/main/assignment2/{filename}?raw=true"
  url = urlopen(base_url)
  binary_data = url.read()
  with open(filename,"wb") as f:
    f.write(binary_data)

def loadDataset():
    parts = ["DLAA2.0.pkl", "DLAA2.1.pkl", "DLAA2.2.pkl", "DLAA2.3.pkl"]
    cData = b''
    for part in parts:
        if not os.path.exists(part):
          download_part(part)
        with open(part, "rb") as f:
            cData += pickle.load(f)
    return pickle.loads(cData)

def addNoise(input, noiseLevel=0.1):
  shape = input.shape
  noise = np.random.randn(*shape)*noiseLevel*np.max(input)
  return input+noise


##Load dataset

In [98]:
x, y = loadDataset()
print(x.shape)
print(y.shape)
print(y[1000])

(2210, 2500, 8)
(2210, 2)
[23 14]


# Your solution

Imports

In [99]:
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

Data preprocessing

In [100]:
total_samples = len(x)
train_ratio=0.7
val_ratio=0.15
test_ratio=0.15

train_size = int(total_samples * train_ratio)
val_size = int(total_samples * val_ratio)
test_size = total_samples - train_size - val_size

# Create random indices for splitting
indices = np.random.permutation(total_samples)

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

x_train, y_train = x[train_indices], y[train_indices]
x_val, y_val = x[val_indices], y[val_indices]
x_test, y_test = x[test_indices], y[test_indices]

scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_val_normalized = scaler.transform(x_val.reshape(-1, x_val.shape[-1])).reshape(x_val.shape)
x_test_normalized = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


Neural network architecture

In [101]:
import torch.nn.functional as F

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(2500 * 8, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.5)
        self.batch_norm1 = nn.BatchNorm1d(2048)
        self.batch_norm2 = nn.BatchNorm1d(1024)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RegressionModel()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
num_epochs = 100

Training the model

In [102]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

    # Print training and validation loss
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader.dataset)}, Val Loss: {val_loss/len(val_loader.dataset)}')

Epoch 1/100, Train Loss: 34.66575196579956, Val Loss: 46.37126061707465
Epoch 2/100, Train Loss: 33.52709227835046, Val Loss: 32.45719500538806
Epoch 3/100, Train Loss: 33.06848721051108, Val Loss: 41.640244060412634
Epoch 4/100, Train Loss: 32.9434565597761, Val Loss: 34.41698128507217
Epoch 5/100, Train Loss: 32.527650076108515, Val Loss: 31.446954853945268
Epoch 6/100, Train Loss: 32.415156156691566, Val Loss: 34.236824508159906
Epoch 7/100, Train Loss: 32.30005969027203, Val Loss: 32.70230152095553
Epoch 8/100, Train Loss: 32.332915452193966, Val Loss: 34.079359440645064
Epoch 9/100, Train Loss: 32.138020951284005, Val Loss: 31.972839539864992
Epoch 10/100, Train Loss: 32.11423394536695, Val Loss: 30.667920847313642
Epoch 11/100, Train Loss: 31.990864711186926, Val Loss: 41.312859906890964
Epoch 12/100, Train Loss: 31.952426811765683, Val Loss: 33.351150748952996
Epoch 13/100, Train Loss: 31.979138840994068, Val Loss: 31.60156842372929
Epoch 14/100, Train Loss: 31.889651761489603, 

Evaluate the model

In [103]:
test_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        test_loss += loss.item() * X_batch.size(0)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')

Test Loss: 30.1496
